In [1]:
import open3d as o3d
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
pcd = o3d.io.read_point_cloud("experiment/test/redandblack/predlift/r01/redandblack.ply.bin.decoded.ply")
print(pcd)
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] Read PLY failed: unable to read file: experiment/test/redandblack/predlift/r01/redandblack.ply.bin.decoded.ply
PointCloud with 757691 points.


In [3]:
sampled=pcd.uniform_down_sample(100)
o3d.visualization.draw_geometries([sampled])
sampled

PointCloud with 7577 points.